In [ ]:
# PyTorch basic code 

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

/anaconda2/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,


In [15]:
# Create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

y = w * x + b

y.backward()

In [16]:
print (x.grad)
print (w.grad)
print (b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


In [21]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)

linear = nn.Linear(3, 2)

In [23]:
print (linear.weight)
print (linear.bias)
print (linear.parameters)

Parameter containing:
tensor([[ 0.1064,  0.0450,  0.0905],
        [ 0.0766, -0.5473, -0.2318]], requires_grad=True)
Parameter containing:
tensor([-0.4468,  0.5110], requires_grad=True)
<bound method Linear.parameters of Linear(in_features=3, out_features=2, bias=True)>


In [24]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

In [25]:
pred = linear(x)

In [28]:
loss = criterion(pred, y)
print (loss)

tensor(1.4510, grad_fn=<MseLossBackward>)


In [29]:
loss.backward()

In [30]:
print ('dL/dw: ', linear.weight.grad)
print ('dL/db: ', linear.bias.grad)

('dL/dw: ', tensor([[ 0.6260,  0.2559, -0.2910],
        [ 0.3595, -0.5437,  0.0958]]))
('dL/db: ', tensor([-0.8131,  0.8673]))


In [32]:
pred = linear(x)
loss = criterion(pred, y)
print ('Loss after 1 step optimization: ', loss.item())

('Loss after 1 step optimization: ', 1.4509841203689575)


In [34]:
x = np.array([[1, 2], [3, 4]])
y = torch.from_numpy(x)
z = y.numpy()

In [35]:
print (x)
print (y)
print (z)

[[1 2]
 [3 4]]
tensor([[1, 2],
        [3, 4]])
[[1 2]
 [3 4]]


In [36]:
# Tutorial 2: Logistic Regression

In [37]:
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [38]:
train_dataset = torchvision.datasets.MNIST(root='../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(), 
                                           download=True)

Processing...
Done!


In [39]:
test_dataset = torchvision.datasets.MNIST(root='../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [40]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=False)

In [41]:
model = nn.Linear(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [43]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Rehape the images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            

Epoch [1/5], Step [100/600], Loss: 2.2212
Epoch [1/5], Step [200/600], Loss: 2.1054
Epoch [1/5], Step [300/600], Loss: 2.0711
Epoch [1/5], Step [400/600], Loss: 1.9754
Epoch [1/5], Step [500/600], Loss: 1.8703
Epoch [1/5], Step [600/600], Loss: 1.7974
Epoch [2/5], Step [100/600], Loss: 1.7842
Epoch [2/5], Step [200/600], Loss: 1.6721
Epoch [2/5], Step [300/600], Loss: 1.6345
Epoch [2/5], Step [400/600], Loss: 1.5703
Epoch [2/5], Step [500/600], Loss: 1.5132
Epoch [2/5], Step [600/600], Loss: 1.4802
Epoch [3/5], Step [100/600], Loss: 1.3725
Epoch [3/5], Step [200/600], Loss: 1.4573
Epoch [3/5], Step [300/600], Loss: 1.3449
Epoch [3/5], Step [400/600], Loss: 1.3838
Epoch [3/5], Step [500/600], Loss: 1.2995
Epoch [3/5], Step [600/600], Loss: 1.2269
Epoch [4/5], Step [100/600], Loss: 1.2358
Epoch [4/5], Step [200/600], Loss: 1.2477
Epoch [4/5], Step [300/600], Loss: 1.1386
Epoch [4/5], Step [400/600], Loss: 1.1630
Epoch [4/5], Step [500/600], Loss: 1.1500
Epoch [4/5], Step [600/600], Loss:

In [44]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    print ('Accuracy of the model on the 10000 test images: {}%'.format(100* correct / total))
    


Accuracy of the model on the 10000 test images: 82%


In [ ]:
# Neural Network Tutorial

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [46]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [47]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3767
Epoch [1/5], Step [200/600], Loss: 0.2777
Epoch [1/5], Step [300/600], Loss: 0.2290
Epoch [1/5], Step [400/600], Loss: 0.1391
Epoch [1/5], Step [500/600], Loss: 0.2500
Epoch [1/5], Step [600/600], Loss: 0.2030
Epoch [2/5], Step [100/600], Loss: 0.0793
Epoch [2/5], Step [200/600], Loss: 0.0755
Epoch [2/5], Step [300/600], Loss: 0.0841
Epoch [2/5], Step [400/600], Loss: 0.0734
Epoch [2/5], Step [500/600], Loss: 0.0945
Epoch [2/5], Step [600/600], Loss: 0.1132
Epoch [3/5], Step [100/600], Loss: 0.0964
Epoch [3/5], Step [200/600], Loss: 0.0834
Epoch [3/5], Step [300/600], Loss: 0.0436
Epoch [3/5], Step [400/600], Loss: 0.0488
Epoch [3/5], Step [500/600], Loss: 0.0633
Epoch [3/5], Step [600/600], Loss: 0.0741
Epoch [4/5], Step [100/600], Loss: 0.0256
Epoch [4/5], Step [200/600], Loss: 0.0443
Epoch [4/5], Step [300/600], Loss: 0.0303
Epoch [4/5], Step [400/600], Loss: 0.0255
Epoch [4/5], Step [500/600], Loss: 0.1046
Epoch [4/5], Step [600/600], Loss:

In [50]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print ('Accuracy of the model on the 10000 test images: {}%'.format(100* correct / total))

Accuracy of the model on the 10000 test images: 98%
